Name : Sayfre Anderson, Howard Wen , Vamsi Sakhamuri

E-mail : ..., ..., vamsi@ischool.berkeley.edu

Class Name : W261-3

Week Number : 3

Date of submission : 02/2/2016

HW3.0.

What is a merge sort? Where is it used in Hadoop?

How is  a combiner function in the context of Hadoop? 

Give an example where it can be used and justify why it should be used in the context of this problem.

What is the Hadoop shuffle?

ANSWER 3.0

* Merge-sort is a sorting algorithm which runs in O(nlog(n)) time. In hadoop, it is used in between the map outputs and 
the reduce inputs. Output from each map output is sorted by the key and then merged with the other sorted outputs from other 
map outputs.

* Combiner function is used after the map output is generated. The values from the same keys are then aggregated to reduce network communication in their transfer to the reduce nodes. Network communication can be a bottleneck and a combiner helps alleviate this problem. 
Whether a combiner function is run or not is entirely dependent on the execution framework. To be totally sure that intermediate map outputs are combined, in-mapper combining function can be employed. This ensures that aggregation will happen 100% before being shuffled across the network to the various reduce nodes.

* An example where combiners (either the optional one provided by the execution framework or the in-mapper combiner) is helpful would be the word count application.
Without any sort of combiners, we would be transfer the same key multiple times within the same map task.

    For example, if the map input is "hello hello hi hi hello hello", 

    Without combiners, the following <key,value> pairs will be emitted across the network and into the reduce nodes:

    hello,1
    
    hello,1
    
    hi,1
    
    hi,1
    
    hello,1
    
    hello,1

    With combiners, the following <key,value> pairs will be emitted across the network and into the reduce nodes:

    hello,4
    
    hi,2

    This is an incredible reduction in network traffic.
    

* Hadoop Shuffle: Map-reduce makes the guarantee that the input to every reducer is sorted by the key. And that the <key,value> pairs with the same key are routed to the same reducer. The process by which the system performs this sort and merge (across various mappers) and transfers the map outputs to the reducers is known as the hadoop shuffle.





<img src="shuffle.png">

HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress)

Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce 
stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the 
jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

In [2]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/start-dfs.sh

16/02/03 21:40:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /Users/Vamsi/Downloads/hadoop-2.7.1/logs/hadoop-Vamsi-namenode-Vamsis-MacBook-Air.local.out
localhost: starting datanode, logging to /Users/Vamsi/Downloads/hadoop-2.7.1/logs/hadoop-Vamsi-datanode-Vamsis-MacBook-Air.local.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /Users/Vamsi/Downloads/hadoop-2.7.1/logs/hadoop-Vamsi-secondarynamenode-Vamsis-MacBook-Air.local.out
16/02/03 21:41:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/mr-jobhistory-daemon.sh --config /Users/Vamsi/Downloads/hadoop-2.7.1/etc/hadoop/ start historyserver 

starting historyserver, logging to /Users/Vamsi/Downloads/hadoop-2.7.1/logs/mapred-Vamsi-historyserver-Vamsis-MacBook-Air.local.out


In [17]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f  /user/hw3

16/02/02 10:30:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -mkdir -p /user/hw3

16/02/03 21:41:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put Consumer_Complaints.csv /user/hw3

16/02/02 10:31:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
#Grabbing the header names
header = next(csv_iter)

for line in csv_iter:
    if(len(line)==len(header)):   #continue only if all the rows have all the entries
        if(line[1].lower()=="debt collection"):
            sys.stderr.write("reporter:counter:Product,Debt,1\n")
        elif(line[1].lower() == "mortgage"):
            sys.stderr.write("reporter:counter:Product,Mortgage,1\n")
        else:
            sys.stderr.write("reporter:counter:Product,Other,1\n")

Overwriting mapper.py


In [394]:
!chmod a+x mapper.py

In [395]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/Consumer_Complaints.csv \
-output /user/hw3/output_3_1 \
-mapper mapper.py \
-reducer org.apache.hadoop.mapred.lib.IdentityReducer 

16/02/01 19:31:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 19:31:13 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 19:31:13 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 19:31:13 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 19:31:14 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 19:31:14 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 19:31:14 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 19:31:14 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local170073006_0001
16/02/01 19:31:14 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/01 19:31:14 INFO mapred.LocalJobRunner: OutputCommitte

<img src="counters.png">

HW 3.2.1 Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.


In [400]:
!echo "foo foo quux labs foo bar quux" >input_3_2

In [401]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put input_3_2 /user/hw3

16/02/01 19:42:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [402]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")   #trigger counter
    line_s = re.split(r'[\s]',line)   #split on whitespace
    for l in line_s:
        print "%s\t1" %l.strip()    #Emit word,1 

Overwriting mapper.py


In [403]:
!chmod a+x mapper.py

In [404]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")  #trigger counter

for line in sys.stdin:
    line_s = re.split(r'[\t]',line)   #split on tab
    
    if((prev_string!=None) and (prev_string != line_s[0])): # Check whether the a new key is emitted
        print prev_string,count  # If a new key is emitted then commit the previous key and count to disk
        count = 0   #reset the counter

    count += 1    #increment the counter
    prev_string = line_s[0] 
print prev_string,count  

Overwriting reducer.py


In [405]:
!chmod a+x reducer.py

In [12]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/hw3/input_3_2 \
-output /user/hw3/output_3_2_1 \
-mapper mapper.py \
-reducer reducer.py 

16/02/02 09:57:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 09:57:48 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/02 09:57:48 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/02 09:57:48 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/02 09:57:48 ERROR streaming.StreamJob: Error Launching job : Call From Vamsis-MacBook-Air.local/10.21.145.49 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
Streaming Command Failed!


<img src="counters_3_2.png">

Following 4 cells would output each of the reducer outputs

In [408]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00000

16/02/01 19:50:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
quux 2	


In [412]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00001 

16/02/01 19:51:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
 1	


In [413]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00002 

16/02/01 19:51:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bar 1	


In [411]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_1/part-00003

16/02/01 19:50:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
labs 1	


HW 3.2.2 Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [1]:
!tail -n +2 Consumer_Complaints.csv > Consumer_Complaints_no_header.csv

In [2]:
!split -l 78228 Consumer_Complaints_no_header.csv

In [9]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put xa* /user/hw3

16/02/02 09:55:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: Call From Vamsis-MacBook-Air.local/10.21.145.49 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused


In [10]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3].strip())      #splitting the issues column
        for w in issues:
            print "%s\t1" %w #Emit word,1 to the reducer 

Overwriting mapper.py


In [11]:
!chmod a+x mapper.py

In [8]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0

    count += 1
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [71]:
!chmod a+x mapper.py

In [310]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/vamsi/hw3/xa* \
-output /user/vamsi/hw3/output_3_2_2 \
-mapper mapper.py \
-reducer reducer.py 

16/02/01 16:26:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:26:20 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 16:26:20 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 16:26:20 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 16:26:21 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/01 16:26:21 INFO mapreduce.JobSubmitter: number of splits:4
16/02/01 16:26:21 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 16:26:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1154581649_0001
16/02/01 16:26:21 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/01 16:26:21 INFO mapred.LocalJobRunner: OutputC

In [311]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00000 | head

16/02/01 17:01:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
repay 1647	
service 1518	
the 6248	
to 8401	
transfer 597	
unable 3821	
verification 7655	
was 274	
wrong 71	
your 3844	


In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00000 | tail

In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00001 | head

In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00001 | tail

In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00002 | head

In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00002 | tail

In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00003 | head

In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_2/part-00003 | tail

HW 3.2.3 Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [314]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3])
        for w in issues:
            print "%s\t1" %w

Overwriting mapper.py


In [ ]:
!chmod a+x mapper.py

In [214]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_2BDC/part-00003

16/02/01 14:48:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Advertising 1193	
Application 8868	
Balance 597	
Charged 878	
Collection 1907	
Customer 2734	
Embezzlement 3276	
Forbearance 350	
Fraud 3842	
Getting 291	
Identity 3276	
Late 1797	
Managing 5006	
Money 274	
Other 7886	
Privacy 240	
Rewards 1002	
Using 2422	
action 2964	
amt 71	
are 3821	
attempts 17972	
charged 98	
collection 70487	
contact 3710	
day 71	
debit 2422	
decrease 1149	
dispute 904	
identity 1453	
increase 1149	
info 3553	
information 29069	
interest 4238	
issues 538	
low 5663	
money 3365	
opening 16205	
pay 3821	
practices 1003	
receiving 3226	
sharing 3489	
stop 131	
when 4095	
with 1944	


In [316]:
%%writefile combiner.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:COMBINER,combiner_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print "%s\t%s" %(prev_string,count)
        count = 0
    count += 1
    prev_string = line_s[0]
print "%s\t%s" %(prev_string,count)

Overwriting combiner.py


In [ ]:
!chmod a+x combiner.py

In [317]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0
    count += int(line_s[1])
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [318]:
!chmod a+x reducer.py

In [319]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/hw3/xa* \
-output /user/hw3/output_3_2_3aaa \
-mapper mapper.py \
-combiner combiner.py \
-reducer reducer.py 

16/02/01 17:05:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:05:16 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:05:16 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:05:16 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:05:16 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/01 17:05:17 INFO mapreduce.JobSubmitter: number of splits:4
16/02/01 17:05:17 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 17:05:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local503180066_0001
16/02/01 17:05:17 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/01 17:05:17 INFO mapred.LocalJobRunner: OutputCommitte

HW 3.2.4 Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [359]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3])
        for w in issues:
            print "%s,1" %w
            print "*,1"

Overwriting mapper.py


In [230]:
!chmod a+x mapper.py

In [360]:
%%writefile combiner.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:COMBINER,combiner_calls,1\n")
    line_s = re.split(r'[,]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print "%s,%s" %(prev_string,count)
        count = 0
    count += 1
    prev_string = line_s[0]
print "%s,%s" %(prev_string,count)

Overwriting combiner.py


In [ ]:
!chmod a+x combiner.py

In [361]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")
    line_s = re.split(r'[,]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0
    count += int(line_s[1])
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [ ]:
!chmod a+x reducer.py

In [362]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/xa* \
-output /user/hw3/output_3_2_4 \
-mapper mapper.py \
-combiner combiner.py \
-reducer reducer.py 

16/02/01 18:43:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:43:13 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 18:43:13 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 18:43:14 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 18:43:14 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 18:43:14 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 18:43:14 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 18:43:14 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local353990645_0001
16/02/01 18:43:15 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/01 18:43:15 INFO mapred.LocalJobRunner: OutputCo

In [363]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_4/part-00000 > reduce_3_2_4

16/02/01 18:44:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [374]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

for line in sys.stdin:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    line = re.split(r'[\s]+',line)
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [373]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
i=0
for line in sys.stdin:
    line = re.split(r'[,]',line.strip())

    if(i==0):
        total_count = int(line[1])
        total_count_read = 1
        i = i+1
    if(total_count_read==1):
        rel_freq = float(line[1])/float(total_count)
        print "%s,%s,%s,%s" %(line[0],line[1],rel_freq,total_count)

Overwriting reducer.py


In [367]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put reduce_3_2_4 /user/hw3

16/02/01 18:45:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [375]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=2 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k2,2nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/reduce_3_2_4 \
-output /user/hw3/output_3_2_4f \
-mapper mapper.py \
-reducer reducer.py

16/02/01 18:54:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:54:11 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 18:54:11 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 18:54:11 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 18:54:12 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 18:54:12 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 18:54:12 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/01 18:54:12 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/01 18:54:12 INFO Configuration.deprecation: mapr

In [376]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_4f/part-00000 | head -n 50

16/02/01 18:54:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
*,1348313,1.0,1348313	
Loan,107254,0.07954681146,1348313	
foreclosure,70487,0.0522779206312,1348313	
collection,70487,0.0522779206312,1348313	
modification,70487,0.0522779206312,1348313	
account,40893,0.0303290111421,1348313	
or,40508,0.0300434691351,1348313	
credit,40483,0.0300249274464,1348313	
payments,39993,0.0296615103466,1348313	
escrow,36767,0.0272688908288,1348313	
servicing,36767,0.0272688908288,1348313	
report,34903,0.0258864225147,1348313	
Incorrect,29133,0.0216070007483,1348313	
on,29069,0.0215595340251,1348313	
information,29069,0.0215595340251,1348313	
debt,26531,0.019677181782,1348313	
not,18477,0.0137037913304,1348313	
owed,17972,0.0133292492174,1348313	
collect,17972,0.0133292492174,1348313	
attempts,17972,0.0133292492174,1348313	
Cont'd,17972,0.0133292492174,1348313	
Account,16555,0.0122783062983,1348313	
and,16448,0.01219

In [378]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_2_4f/part-00000 | tail -n 10

16/02/01 18:57:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
credited,92,6.82334146448e-05,1348313	
checks,75,5.56250662865e-05,1348313	
Convenience,75,5.56250662865e-05,1348313	
wrong,71,5.26583960846e-05,1348313	
day,71,5.26583960846e-05,1348313	
amt,71,5.26583960846e-05,1348313	
disclosures,64,4.74667232312e-05,1348313	
missing,64,4.74667232312e-05,1348313	
,4,2.96667020195e-06,1348313	
Issue,1,7.41667550487e-07,1348313	


HW3.3. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

In [5]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put ProductPurchaseData.txt /user/hw3

16/02/03 21:41:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    line = re.split(r'[\s]',line.strip())
    for l in line:
        print "%s\t1" %l
        print "*\t1"

Overwriting mapper.py


In [7]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

count = 0
prev_id = None

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    if((prev_id !=None) and (line[0] !=prev_id)):
        print "%s\t%s" %(prev_id,count)
        count = 0
    count +=1
    prev_id = line[0]
print "%s\t%s" %(prev_id,count)

Overwriting reducer.py


In [8]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/ProductPurchaseData.txt \
-output /user/hw3/output_3_3n33 \
-mapper mapper.py \
-reducer reducer.py 

16/02/03 21:41:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 21:41:53 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 21:41:53 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 21:41:53 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 21:41:54 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 21:41:54 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 21:41:54 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/03 21:41:54 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1683492560_0001
16/02/03 21:41:54 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/03 21:41:54 INFO mapred.LocalJobRunner: OutputC

In [9]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_3n33/part-00000 > counts_3_3

16/02/03 21:42:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Number of unique items is 12593

In [10]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put counts_3_3 /user/hw3

16/02/03 21:42:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [32]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

i = 0
for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [33]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
i=0
for line in sys.stdin:
    line = re.split(r'[,]',line.strip())

    if(i==0):
        total_count = int(line[1])
        i = i+1
    elif(i==1):
        rel_freq = float(line[1])/float(total_count)
        print "%s,%s,%s,%s" %(line[0],line[1],rel_freq,total_count)


Overwriting reducer.py


In [34]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=2 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k2,2nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/counts_3_3 \
-output /user/hw3/output_3_3_out14 \
-mapper mapper.py \
-reducer reducer.py

16/02/03 22:01:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/03 22:01:34 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/03 22:01:34 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/03 22:01:34 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/03 22:01:35 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/03 22:01:35 INFO mapreduce.JobSubmitter: number of splits:1
16/02/03 22:01:35 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/03 22:01:35 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/03 22:01:35 INFO Configuration.deprecation: mapr

In [35]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_3_out14/part-00000 | wc -l 

16/02/03 22:01:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
   12592


In [36]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_3_out14/part-00000 | head -n 50

16/02/03 22:01:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779,6667,0.0175067747831,380824	
FRO40251,3881,0.010191059387,380824	
ELE17451,3875,0.0101753040775,380824	
GRO73461,3602,0.00945843749344,380824	
SNA80324,3044,0.00799319370628,380824	
ELE32164,2851,0.0074863979161,380824	
DAI75645,2736,0.00718442114993,380824	
SNA45677,2455,0.0064465474865,380824	
FRO31317,2330,0.0061183118711,380824	
DAI85309,2293,0.00602115412894,380824	
ELE26917,2292,0.00601852824402,380824	
FRO80039,2233,0.00586360103355,380824	
GRO21487,2115,0.00555374661261,380824	
SNA99873,2083,0.00546971829507,380824	
GRO59710,2004,0.00526227338613,380824	
GRO71621,1920,0.00504169905258,380824	
FRO85978,1918,0.00503644728273,380824	
GRO30386,1840,0.00483162825872,380824	
ELE74009,1816,0.00476860702057,380824	
GRO56726,1784,0.00468457870302,380824	
DAI63921,1773,0.00465569396887,380824	
GRO46854,1756,0.00461105392517,380824	
E

HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

In [316]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    line = re.split(r'[\s]',line.strip())
    for l in line:
        for p in line:
            if(l != p):
                print "%s,%s\t1" %(l,p)
                #print "%s,*\t1" %l   Use this only if you want total number of tuples in the denominator for support
    print "BASKET,*\t1" #Use this if you want basket in the denominator for support calculation

Overwriting mapper.py


In [317]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

count = 0
prev_id = None

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    if((prev_id !=None) and (line[0] !=prev_id)):
        print "%s\t%s" %(prev_id,count)
        count = 0
    count +=1
    prev_id = line[0]
print "%s\t%s" %(prev_id,count)

Overwriting reducer.py


In [318]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/ProductPurchaseData.txt \
-output /user/hw3/output_3_4_2_1 \
-mapper mapper.py \
-reducer reducer.py 

16/02/02 16:18:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 16:18:55 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/02 16:18:55 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/02 16:18:55 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/02 16:18:55 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/02 16:18:55 INFO mapreduce.JobSubmitter: number of splits:1
16/02/02 16:18:55 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/02 16:18:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1637429785_0001
16/02/02 16:18:56 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/02 16:18:56 INFO mapred.LocalJobRunner: OutputC

In [330]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_4_2_1/part-00000 > pairs_unsorted

16/02/02 16:32:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [332]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put pairs_unsorted /user/hw3

16/02/02 16:32:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [417]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [416]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    line = re.split(r'[,]',line.strip())
    if(line[1]=="*"):
        total = int(line[2]) 
    else:
        rel_freq = float(line[2])/float(total)
        print "%s,%s,%s,%s,%s" %(line[0],line[1],line[2],rel_freq,total)

Overwriting reducer.py


In [335]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=3 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k3,3nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/pairs_unsorted \
-output /user/hw3/output_3_4_o4 \
-mapper mapper.py \
-reducer reducer.py

16/02/02 16:33:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 16:33:39 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/02 16:33:39 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/02 16:33:39 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/02 16:33:40 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/02 16:33:40 INFO mapreduce.JobSubmitter: number of splits:1
16/02/02 16:33:40 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/02 16:33:40 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/02 16:33:40 INFO Configuration.deprecation: mapr

In [338]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_4_o4/part-00000 | head -n 50

16/02/02 16:35:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779,ELE17451,1592,0.0511880646925,31101	
ELE17451,DAI62779,1592,0.0511880646925,31101	
FRO40251,SNA80324,1412,0.0454004694383,31101	
SNA80324,FRO40251,1412,0.0454004694383,31101	
DAI75645,FRO40251,1254,0.0403202469374,31101	
FRO40251,DAI75645,1254,0.0403202469374,31101	
GRO85051,FRO40251,1213,0.0390019613517,31101	
FRO40251,GRO85051,1213,0.0390019613517,31101	
GRO73461,DAI62779,1139,0.0366226166361,31101	
DAI62779,GRO73461,1139,0.0366226166361,31101	
SNA80324,DAI75645,1130,0.0363332368734,31101	
DAI75645,SNA80324,1130,0.0363332368734,31101	
DAI62779,FRO40251,1070,0.0344040384554,31101	
FRO40251,DAI62779,1070,0.0344040384554,31101	
DAI62779,SNA80324,923,0.0296775023311,31101	
SNA80324,DAI62779,923,0.0296775023311,31101	
DAI85309,DAI62779,918,0.0295167357963,31101	
DAI62779,DAI85309,918,0.0295167357963,31101	
GRO59710,ELE32164,911,0.0292

HW3.5: Stripes
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs

In [419]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    line = re.split(r'[\s]',line.strip())
    
    for l in line:
        co_stripe = {}
        for p in line:
            if(l != p):
                if p not in co_stripe.keys():
                    co_stripe[p] = 1 
                else:
                    co_stripe[p] += 1
        print "%s\t%s" %(l,co_stripe)
                #print "%s,*\t1" %l   Use this only if you want total number of tuples in the denominator for support
    print "BASKET\t{'*':1}" #Use this if you want basket in the denominator for support calculation

Overwriting mapper.py


In [420]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import ast

count = {}
prev_id = None

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    co_strip = ast.literal_eval(line[1])
    if((prev_id !=None) and (line[0] !=prev_id)):
        for w in count.keys():
            print "%s,%s\t%s" %(prev_id,w,count[w])
        count = {}
    for w in co_strip:
        if w in count.keys():
            count[w] +=1
        else:
            count[w] = 1
    prev_id = line[0]

for w in count.keys():
    print "%s,%s\t%s" %(prev_id,w,count[w])

Overwriting reducer.py


In [421]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/hw3/ProductPurchaseData.txt \
-output /user/hw3/output_3_5_12 \
-mapper mapper.py \
-reducer reducer.py 

16/02/02 19:09:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 19:09:43 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/02 19:09:43 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/02 19:09:43 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/02 19:09:44 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/02 19:09:44 INFO mapreduce.JobSubmitter: number of splits:1
16/02/02 19:09:44 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/02 19:09:44 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local667678344_0001
16/02/02 19:09:44 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/02 19:09:44 INFO mapreduce.Job: Running job: job

In [422]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_5_12/part-00000 > stripes_unsorted

16/02/02 19:13:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [429]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put stripes_unsorted /user/hw3

16/02/02 19:16:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [430]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    line = re.split(r'[\t]',line.strip())
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [431]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    line = re.split(r'[,]',line.strip())
    if(line[1]=="*"):
        total = int(line[2]) 
    else:
        rel_freq = float(line[2])/float(total)
        print "%s,%s,%s,%s,%s" %(line[0],line[1],line[2],rel_freq,total)

Overwriting reducer.py


In [432]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=3 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k3,3nr \
-D mapred.reduce.tasks=1 \
-input /user/hw3/stripes_unsorted \
-output /user/hw3/output_3_5_o11 \
-mapper mapper.py \
-reducer reducer.py

16/02/02 19:16:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 19:16:21 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/02 19:16:21 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/02 19:16:21 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/02 19:16:21 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/02 19:16:21 INFO mapreduce.JobSubmitter: number of splits:1
16/02/02 19:16:21 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/02 19:16:21 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/02 19:16:21 INFO Configuration.deprecation: mapr

In [434]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/hw3/output_3_5_o11/part-00000 | head -n 50

16/02/02 19:17:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779,ELE17451,1592,0.0511880646925,31101	
ELE17451,DAI62779,1592,0.0511880646925,31101	
FRO40251,SNA80324,1412,0.0454004694383,31101	
SNA80324,FRO40251,1412,0.0454004694383,31101	
DAI75645,FRO40251,1254,0.0403202469374,31101	
FRO40251,DAI75645,1254,0.0403202469374,31101	
FRO40251,GRO85051,1213,0.0390019613517,31101	
GRO85051,FRO40251,1213,0.0390019613517,31101	
GRO73461,DAI62779,1139,0.0366226166361,31101	
DAI62779,GRO73461,1139,0.0366226166361,31101	
SNA80324,DAI75645,1130,0.0363332368734,31101	
DAI75645,SNA80324,1130,0.0363332368734,31101	
DAI62779,FRO40251,1070,0.0344040384554,31101	
FRO40251,DAI62779,1070,0.0344040384554,31101	
SNA80324,DAI62779,923,0.0296775023311,31101	
DAI62779,SNA80324,923,0.0296775023311,31101	
DAI62779,DAI85309,918,0.0295167357963,31101	
DAI85309,DAI62779,918,0.0295167357963,31101	
GRO59710,ELE32164,911,0.0292

In [61]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/stop-dfs.sh

16/01/31 17:22:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: no datanode to stop
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/31 17:22:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
